In [7]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import DirectoryLoader

In [ ]:
!pip install -U openai

In [9]:
import openai

In [10]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = ""

In [11]:
# Read questions from question.txt file and extract them
questions = []
with open("evaluation/evaluation.txt", "r") as file:
    lines = file.readlines()
    for line in lines:
        # Extract questions after the question number
        question = line.strip()
        questions.append(question)

In [12]:
for question in questions:
    print(question+"\n")

Could you please share your full name and age with me?

Can you tell me a bit about your family background? Do you have any other family members at home?

What do you typically enjoy doing in your daily life? Do you have any particular interests or hobbies?

I'd like to learn about your school life. Which school do you attend, and how do you feel about it?

Do you have any specific career or vocational goals? What kind of work would you like to do in the future?

Could you tell me about your friends or social circle? Do you have any close friends?

Have you experienced any past mental health issues or therapy? Do you feel these experiences have any impact on your current state?

How would you describe your emotional and mood states in your daily life? Do you sometimes feel low or anxious?

Let's talk about your family relationships. How would you describe your relationships with your family members? Do you feel family dynamics affect your mood?

Lastly, Tom, what do you hope to gain fr

In [16]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4")

In [23]:
# Load, chunk and index the contents of the blog.
loader = DirectoryLoader("RAG_dataset", glob="*.txt")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=900, chunk_overlap=0)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
Your name is Tom and you are now communicating with a psychologist in a therapy session.
Answer the following questions as Tom and respond in a casual tone. Each of your replies should be no longer than three sentences.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)


In [6]:
rag_chain.invoke("### Human: How old are you? ### Assistant: ")

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


"I'm 15 years old."

In [24]:
with open("evaluation/gpt_rag-900.txt", "a") as output_file:
    for question in questions:
        response = rag_chain.invoke(f"### Human: {question} ### Assistant: ")
        output_file.write(response + "\n")